[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cjaniake/xsum_llm_eval/blob/main/docs/getting_started.ipynb)

# Evaluating LLM applications